In [ ]:
# System
import os
from collections import defaultdict, Counter
import csv
import re
import progressbar

# Data Analysis
import pandas
import tabulate
import seaborn
import numpy
import matplotlib.pyplot as plt
import matplotlib as mpl

# Warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

# Short hack for display of images in jupyter notebook

from IPython.display import display, HTML
display(HTML("<style>div.output_area pre {white-space: pre;}</style>"))

In [ ]:
FIGS_DIR = "figs"
def save_figure(fig, ext=".png"):
    """Stores @p fig at `figs/fig.@ext`"""
    tgt_dir = os.path.join(DATA_SOURCE, FIGS_DIR)
    tgt = os.path.join(tgt_dir, fig + ext)    
    if not os.path.exists(tgt_dir):
        os.makedirs(tgt_dir)
    print(f"Saving to {tgt}")
    if ext == ".png":
        plt.savefig(tgt, backend="cairo", bbox_inches="tight", pad_inches=0.2)
    else:
        plt.savefig(tgt, bbox_inches="tight", pad_inches=0.2)

In [ ]:
DATA_DIR = "./data/"
for experiment_dir in os.listdir(DATA_DIR):
    full_path = os.path.join(DATA_DIR, experiment_dir)
    if os.path.isdir(full_path):
        print(f'DATA_SOURCE = "{full_path}"')

In [ ]:
DATA_SOURCE = "./data/experiments-10-07"

## Creating DataFrame

In [ ]:
ERR = 60
TIMEOUT = 60

def to_operation(src, lang):
    op = src.split('-')[-1]
    if 'runtime' in op:
        return 'runtime'
    elif 'overall' in op:
        return 'overall'
    elif 'interpretation' in op:
        return 'interpretation'
    elif 'deter' in op:
        return 'determization'
    elif 'minterm' in op:
        return 'minterm'
    elif 'reduce' in op:
        return 'reduce'
    elif 'inter' in op:
        return 'intersection'
    elif 'union' in op or 'uni' in op:
        return 'union'
    elif ('construction' in op and lang != 'c++') or 'parsing' in op:
        return 'parsing'
    elif ('construction' in op and lang == 'c++') or 'conversion' in op:
        return 'transform'
    elif 'concat' in op:
        return 'concatenation'
    elif 'result' in op:
        return 'result'
    elif 'inclusion_check' == op or 'inclusion' == op:
        return 'inclusion'
    elif 'emptiness_check' == op or 'emptiness' == op:
        return 'emptiness'
    elif 'compl' == op or 'complementation' == op or 'complement' == op:
        return 'complement'
    elif 'trim' in op:
        return 'trim'
    print(f"{src} unhandled")
    assert False

def to_tool_and_lang(tool):
    if 'mata-bin' in tool or 'stats' in tool:
        return None, None
    elif 'mata-0.111' in tool:
        return "mata-0.111", "c++"
    elif 'mata-0.112' in tool:
        return "mata-0.112", "c++"
    elif 'mata-0.113' in tool:
        return "mata-0.113", "c++"
    elif 'mata-sim' in tool:
        return "mata-sim", "c++"
    elif 'mata-old' in tool:
        return "mata-old", "c++"
    elif 'awali' in tool:
        return 'awali', 'c++'
    elif 'mona' in tool:
        return 'mona', 'c++'
    elif 'vata' in tool:
        return 'vata', 'c++'
    elif 'java-brics' in tool:
        return 'brics', 'java'
    elif 'java-automata' in tool:
        return '(j)alib', 'java'
    elif 'pyfado' in tool:
        return 'fado', 'python'
    elif 'pyautomata-lib' in tool:
        return '(py)alib', 'python'
    elif 'pymata' in tool:
        return '(py)mata', 'python'
    elif 'automata' in tool:
        return 'automata', 'c#'
    elif 'mata' in tool:
        return 'mata', 'c++'
    print(f"{tool} unhandled")
    assert False

def to_bench(bench, src):
    if 'automata_inclusion' in bench:
        return 'aut_inclusion'
    elif 'comb/ere' in bench:
        return 'bc_ere'
    elif ('cox/diff' in bench or 'cox/inter' in bench) and 'union' in src :
        return 'bc_cox_union'
    elif ('cox/diff' in bench or 'cox/inter' in bench) and 'cox-inter' in src :
        return 'bc_cox_inter'
    elif ('cox/diff' in bench or 'cox/inter' in bench) and 'cox-diff' in src :
        return 'bc_cox_diff'
    elif 'email_filter' in bench:
        return 'email_filter'
    elif 'z3-noodler' in bench:
        if 'concat' in src:
            return 'z3_noodler_concat'
        elif 'intersect' in src:
            return 'z3_noodler_intersect'
        else:
            return 'z3_noodler'
    elif 'presburger-explicit' in bench:
        return 'presburger-explicit'
    elif 'presburger' in bench:
        return 'presburger'
    elif 'intersect' in bench:
        return 'bc_intersect' if 'union' not in src else 'bc_intersect_union'
    print(f"{bench} unhandled")
    assert False

def to_value(val):
    val = val.strip()
    try:
        return float(val)
    except ValueError:
        pass
    if val in ['EMPTY', "NOT EMPTY"]:
        return val
    elif val in ('false', 'False'):
        return 'false'
    elif val in ('true', 'True'):
        return 'true'
    elif val == 'ERR':
        return 'ERR'
    elif val == 'MISSING':
        return numpy.NAN
    elif val == 'TIMEOUT' or val == 'TO':
        return TIMEOUT
    print(f"{val} unhandled")
    assert False

In [ ]:
HEADERS = ["bench", "input", "tool", "lang", "op", "time"]
TIMEOUT = 60
TIMEOUT_REGEX = re.compile("timeout-(\d+)")
processed = defaultdict(set)
op_map = defaultdict(list)
bench_map = defaultdict(set)
def to_pandas(src_dir):
    global TIMEOUT
    data = []
    for csv_source in progressbar.progressbar(os.listdir(src_dir)):
        if csv_source.endswith('.csv'):
            if timeout := TIMEOUT_REGEX.search(csv_source):
                TIMEOUT = int(timeout.group(1))
            with open(os.path.join(src_dir, csv_source), 'r', newline='') as csvfile:
                try:
                    csv_reader = csv.reader(csvfile, delimiter=';')
                    head = next(csv_reader)
                    for row in csv_reader:
                        bench = to_bench(row[0], csv_source) # bench
                        bench_map[(row[0], bench)].add(csv_source)
                        inputs = row[0] # inputs
                        for i, val in enumerate(row[1:], 1):
                            tool, lang = to_tool_and_lang(head[i]) # tool, lang
                            if not tool:
                                continue
                            op = to_operation(head[i], lang) # op
                            op_map[op].append(head[i])
                            val = to_value(val)
                            data.append([bench, inputs, tool, lang, op, val])
                except StopIteration:
                    pass
    return pandas.DataFrame(data, columns=HEADERS)
df = to_pandas(DATA_SOURCE)
print(df)

In [ ]:
data = {
    'bench': [],
    'input': [],
    'tool': [],
    'lang': [],
    'op': [],
    'time': []
}
def to_float(val, default=0):
    vals = list(val)
    if val.empty:
        if default == None:
            print(f"{val=}, {vals=}")
            assert False
        return default
    if len(vals) != 1:
        print(f"{vals=}")
        assert False
    try:
        return float(vals[0])
    except:
        return 0 if str(vals[0]) not in ('ERR', 'TIMEOUT') else TIMEOUT
for grp, series in df.groupby(['bench', 'input', 'tool']):
    data['bench'].append(grp[0])
    data['input'].append(grp[1])
    data['tool'].append(grp[2])
    data['lang'].append(list(series['lang'])[0])
    data['op'].append('fair-overall')

    pyco_runtime = list(series[series['op'] == 'runtime']['time'])
    if len(pyco_runtime) != 1:
        print(f"{list(series.items())=}")
        print(f"{pyco_runtime=}")
        assert False
    if pyco_runtime[0] == TIMEOUT or pyco_runtime[0] == 'ERR':
        data['time'].append(TIMEOUT)
        continue
        
    
    runtime = to_float(series[series['op'] == 'overall']['time'], None)
    parsing = to_float(series[series['op'] == 'parsing']['time'], 0)
    transform = to_float(series[series['op'] == 'transform']['time'], 0)
    fair_runtime = runtime - parsing - transform
    if grp[2] == 'mona':
        data['time'].append(runtime)
    else:
        data['time'].append(fair_runtime)
ddf = pandas.DataFrame(data)
df = pandas.concat([df, ddf])

In [ ]:
def sum_generator(series, timeout=None):
    """Cumulatively sums the @p series wrt @p timeout"""
    sum = 0
    series = sorted(
        [a if isinstance(a, float) or isinstance(a, int) else numpy.NAN for a in series['time']],
        key = lambda x: float('inf') if numpy.isnan(x) else x
    )
    for num in sorted(series):
        if numpy.isnan(num):
            yield None
        if timeout and num >= timeout:
            yield None
        else:
            sum += num
            yield sum

In [ ]:
tools = sorted(t for t in set(df['tool']))
tool_len = len(tools)
color_map = {
    t: c for (t, c) in zip(tools, mpl.colormaps['tab20'].resampled(tool_len).colors)
}

### Final visualizations

In [ ]:
bench_list = sorted(list(set(list(df['bench']))))
item_no = len(bench_list)
x_dim = item_no // 3 + 1
y_dim = min(item_no, 3)
for op in ('runtime', 'fair-overall'):
    fig, ax = plt.subplots(x_dim, y_dim, figsize=(x_dim * 5, y_dim * 6))
    plt.subplots_adjust(top=0.99, bottom=0.01, hspace=0.2, wspace=0.10)
    
    for grp in bench_list:
        series = df[df['bench'] == grp]
        series = series[series['op'] == 'fair-overall']
        i = bench_list.index(grp)
        grp_name = f"{grp}"
        
        data = {}
        for tool, values in series.groupby('tool'):
            data[tool] = list(sum_generator(values, timeout=TIMEOUT))
            
        g = seaborn.lineplot(
            data, linewidth=3.5, palette=color_map, dashes="", ax=ax[i // 3, i % 3] if item_no > 1 else ax
        )
        
        g.set(yscale="linear")
        g.set_xticklabels(g.get_xticklabels(), rotation=30)
        g.set_title(f"{grp}", x=0.05)
        if i % 3 == 0:
            g.set_ylabel("time [s]")
        if i // 3 == 2:
            g.set_xlabel("benchmark")
        
        seaborn.move_legend(g, "upper left", bbox_to_anchor=(0., 1), frameon=True)
        x_lim_min, x_lim_max = g.get_xlim()
        g.set_xlim((x_lim_min, x_lim_max))
    
    save_figure(f"paper-cactus-plot-per-bench-{op}")

In [ ]:
op_list = sorted(list(set(list(df['op']))))
print(f"available: {op_list}")
op_list = [
    'complement', 'concatenation', 'determization', 
    'emptiness', 'fair-overall', 'inclusion', 
    'interpretation', 'intersection', 'minterm', 
    'overall', 'parsing', 'reduce', 
    'runtime', 'transform', 'trim', 'union'
]
item_no = len(op_list)
x_dim = item_no // 3 + 1
y_dim = min(item_no, 3)
fig, ax = plt.subplots(x_dim, y_dim, figsize=(x_dim * 5, y_dim * 6))
plt.subplots_adjust(top=0.99, bottom=0.01, hspace=0.2, wspace=0.10)

for grp in op_list:
    series = df[df['op'] == grp]
    i = op_list.index(grp)
    grp_name = f"{grp}"
    
    data = {}
    for tool, values in series.groupby('tool'):
        data[tool] = list(sum_generator(values, timeout=TIMEOUT))
        
    g = seaborn.lineplot(
        data, linewidth=3.5, palette=color_map, dashes="", ax=ax[i // 3, i % 3] if item_no > 1 else ax
    )
    
    g.set(yscale="logit")
    g.set_xticklabels(g.get_xticklabels(), rotation=30)
    g.set_title(f"{grp}", x=0.05)
    if i % 3 == 0:
        g.set_ylabel("time [s]")
    if i // 3 == 2:
        g.set_xlabel("benchmark")
    
    seaborn.move_legend(g, "upper left", bbox_to_anchor=(0., 1), frameon=True)
    x_lim_min, x_lim_max = g.get_xlim()
    g.set_xlim((x_lim_min, x_lim_max))

save_figure(f"paper-cactus-plot-per-operation")

### Rest of the visualizations

In [ ]:
def to_table(df, rows, aggregation, trim_rows=False, trimsize=5):
    tools = ['mata', 'mata-0.111', 'mata-0.112', 'mata-0.113', 'mata-old', 'mata-sim', 'awali', 'mona', 'vata',  'automata', 'brics', '(j)alib', 'fado', '(py)alib', '(py)mata']
    tools = [t for t in tools if t in set(df['tool'])]
    data = {
        grp: [grp[:trimsize] if trim_rows else grp] + ['-' for i in range(len(tools))] for grp in set(df[rows]) if grp != 'result' and 'result' not in grp
    }
    for grp, series in df.groupby([rows, 'tool'] if not isinstance(rows, list) else rows + tools):
        if grp[0] == 'result' or 'result' in grp[0]:
            continue
        vals = aggregation(series['time'])
        data[grp[0]][tools.index(grp[1]) + 1] = ", ".join(vals)
    return tabulate.tabulate(
        sorted(data.values()), headers=[rows] + tools
    )

def to_printable_table(table, title):
    table_len = len(table.split('\n')[1])
    printable = title.center(table_len) + "\n"
    printable += "-" * table_len + "\n"
    printable += table
    return printable

def print_table(table, title):
    printable = to_printable_table(table, title)
    print(printable)

def save_table(table, title, filename):
    tgt_dir = os.path.join(DATA_SOURCE, FIGS_DIR)
    tgt = os.path.join(tgt_dir, filename)    
    printable = to_printable_table(table, title)
    if not os.path.exists(tgt_dir):
        os.makedirs(tgt_dir)
    with open(tgt, 'w') as table_h:
        table_h.write(printable)
    print(f"Saved to {tgt}")
    

In [ ]:
## Mean, Median Mean with timeouts/errors
def mean_med_overall(series):
    times = [t for t in series if  (isinstance(t, float) or isinstance(t, int)) and t >= 0 and t < TIMEOUT]
    times_with_timeout = [t if (isinstance(t, float) or isinstance(t, int)) and t >= 0 else TIMEOUT for t in series]
    mean = round(numpy.mean(times or [-1]), 1)
    mean_with_timeouts = round(numpy.mean(times_with_timeout or [-1]), 1)
    median = round(numpy.median(times or [-1]), 1)
    return (
        f"{mean:0.01f}" if mean != -1 else f"{'-':<4}", 
        f"{median:0.1f}" if median != -1 else f"{'-':<5}", 
        f"{mean_with_timeouts:0.1f}" if mean != -1 else f"{'-':<4}"
    )
table = to_table(df, 'op', mean_med_overall, trim_rows=True)
print_table(table, "Average / Median / Average with Timeouts")
print()
save_table(table, "Average / Median / Average with Timeouts", "avg-med-avgt.txt")

In [ ]:
## Finished, Errors and Timeouts
def fin_err_tim(series):
    timeouts = [a for a in series if (isinstance(a, float) or isinstance(a, int)) and a >= TIMEOUT and not numpy.isnan(a) and a != 'ERR']
    errors = [a for a in series if a == 'ERR']
    times = [t for t in series if  (isinstance(t, float) or isinstance(t, int)) and t >= 0 and t < TIMEOUT]
    return (
        f"{len(times):}",
        f"{len(errors):}",
        f"{len(timeouts):}"
    )
table = to_table(df[df['op'] == 'runtime'], 'bench', fin_err_tim)
print_table(table, "Finished / Errors / Timeouts")
print()
save_table(table, "Finished / Errors / Timeouts", "fin-err-to.txt")

In [ ]:
## First, Second, Third Quartiles
def quartiles(series):
    times = [t if (isinstance(t, float) or isinstance(t, int)) and t >= 0 and t < TIMEOUT else TIMEOUT for t in series]
    first = numpy.quantile(times, 0.25)
    second = numpy.quantile(times, 0.5)
    third = numpy.quantile(times, 0.75)
    iqr = third - first
    return (
        f"{first:0.0f}" if first < TIMEOUT else f"{TIMEOUT}", 
        f"{second:0.0f}" if second < TIMEOUT else f"{TIMEOUT}", 
        f"{third:0.0f}" if third < TIMEOUT else f"{TIMEOUT}"
    )
table = to_table(df, 'bench', quartiles, trim_rows=True)
print_table(table, "1st Quartile / Median / 3rd Quartile")
print()
save_table(table, "1st Quartile / Median / 3rd Quartile", "quartiles.txt")

In [ ]:
## First, Second, Third Quartiles
def quartiles(series):
    times = [t if (isinstance(t, float) or isinstance(t, int)) and t >= 0 and t < TIMEOUT else TIMEOUT for t in series]
    first = numpy.quantile(times, 0.25)
    second = numpy.quantile(times, 0.5)
    third = numpy.quantile(times, 0.75)
    iqr = third - first
    return (
        f"{first:0.1f}" if first < TIMEOUT else f"{TIMEOUT}", 
        f"{second:0.1f}" if second < TIMEOUT else f"{TIMEOUT}", 
        f"{third:0.1f}" if third < TIMEOUT else f"{TIMEOUT}"
    )
table = to_table(df, 'op', quartiles, trim_rows=True)
print_table(table, "1st Quartile / Median / 3rd Quartile per operation")
print()
save_table(table, "1st Quartile / Median / 3rd Quartile per operation", "quartiles-op.txt")

In [ ]:
## First, Second, Third Quartiles
def quartiles(series):
    times = [t if (isinstance(t, float) or isinstance(t, int)) and t >= 0 and t < TIMEOUT else TIMEOUT for t in series]
    first = numpy.quantile(times, 0.25)
    second = numpy.quantile(times, 0.5)
    third = numpy.quantile(times, 0.75)
    iqr = third - first
    return (
        f"{first:0.1f}" if first < TIMEOUT else f"{TIMEOUT}", 
        f"{second:0.1f}" if second < TIMEOUT else f"{TIMEOUT}", 
        f"{third:0.1f}" if third < TIMEOUT else f"{TIMEOUT}"
    )
df['benchop'] = [f"{o[:5]}/{''.join(a[0] for a in b.split('_'))}" for b, o in zip(df['bench'], df['op'])]
table = to_table(df, 'benchop', quartiles, trim_rows=True, trimsize=9)
print_table(table, "1st Quartile / Median / 3rd Quartile per op/bench")
print()
save_table(table, "1st Quartile / Median / 3rd Quartile per op/bench", "quartiles-benchop.txt")

In [ ]:
print("DONE")